In [1]:
#selenium ver4とChromeのドライバをインストール
!pip -q install selenium==4.4.3
!pip -q install webdriver-manager

You should consider upgrading via the 'C:\Users\yusuke\Anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'C:\Users\yusuke\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
#selenium関連
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

#データフレーム関連
import pandas as pd

In [3]:
#ドライバのインストール(Chrome)
service = ChromeService(executable_path=ChromeDriverManager().install())

#Headlessモードの指定
options = Options()
options.headless = True

#ドライバ起動
driver = webdriver.Chrome(service=service)

#サーバ負荷対策
driver.implicitly_wait(0.5)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 6.68M/6.68M [00:00<00:00, 21.7MB/s]


In [322]:
#外生変数の設定

#ブランド名
product_brand = "sample" #String型で指定
#保存気温
product_temperature = "Ambient" #String型で指定
#Supermarket
product_supermarket = "sample" #String型で指定

#ページの読込
root = 'Webスクレイピング対象HPのTOP画面URLを記載' #TOP 

#空の辞書を定義(取得データ格納先)
data_product = {
    'Brand': [], 
    'SKU_NAME_JP': [], 
    'SKU_Barcode': [], 
    'SKU_Weight_(g/unit)': [], 
    'Consumer_Price': [], 
    'Temperature': [], 
    'Supermarket': [], 
    'Detail':[],
    'Raw_Material':[],
}

In [323]:
#Topページへ遷移
driver.get(root)

#カテゴリページurl格納先
category_url_list = []

#カウンターインデックスの初期化
counter = 0

#カテゴリ一覧の取得
category_content = driver.find_element(By.CLASS_NAME,"index-04.col5")
category_lists = category_content.find_elements(By.TAG_NAME,"li")

for category_list in category_lists:
    #各カテゴリのliタグ情報取得
    category_li_tag_content = category_list.get_attribute('innerHTML')
    
    #liタグ情報よりカテゴリ単体のaタグ情報を取得し、URLを抽出
    category_a_tag_url = category_li_tag_content.split('"')[1]
    
    #urlの文字実体参照(amp;)への対応
    category_a_tag_url_no_amp = category_a_tag_url.replace('amp;', '')

    #カテゴリページへのurlをリストへ格納
    category_url_list.append(category_a_tag_url_no_amp)

#業務用ページを除外 (データ取得不可のため)
new_category_url_list = [category_url for category_url in category_url_list if "prouse.html" not in category_url]


#トップページ
for category_url in new_category_url_list:
    if "brand" not in category_url: #Category_urlの中にbrandが含まれない場合、サブカテゴリ一覧ページ有り:
        target_category_url = root + category_url
        driver.get(target_category_url)
        #サブカテゴリページurl格納先
        sub_category_url_list = []
        #サブカテゴリ一覧の取得
        sub_category_content = driver.find_element(By.CLASS_NAME,"content-inner")
        sub_category_lists = sub_category_content.find_elements(By.TAG_NAME,"li")

        for sub_category_list in sub_category_lists:
            #各サブカテゴリのliタグ情報取得
            sub_category_li_tag_content = sub_category_list.get_attribute('innerHTML')
            #liタグ情報よりサブカテゴリ単体のaタグ情報を取得し、URLを抽出
            sub_category_a_tag_url = sub_category_li_tag_content.split('"')[1]
            #urlの文字実体参照(amp;)への対応
            sub_category_a_tag_url_no_amp = sub_category_a_tag_url.replace('amp;', '')
            #サブカテゴリページへのurlをリストへ格納
            sub_category_url_list.append(sub_category_a_tag_url_no_amp)
    
    else:  #Category_urlの中にbrandが含まれる場合、サブカテゴリ一覧ページ無し
        sub_category_url_list = []
        sub_category_url_list.append(category_url)
        
    #サブカテゴリページ
    for sub_category_url in sub_category_url_list:
        target_sub_category_url = root + sub_category_url
        driver.get(target_sub_category_url)

        #商品詳細ページurl格納先
        product_url_list = []

        #各ジャンルの一覧を取得
        product_lists = driver.find_elements(By.CLASS_NAME, "show")

        for product_list in product_lists:
            #各商品のliタグ情報取得
            li_tag_contents = product_list.find_elements(By.TAG_NAME,"li")

            #liタグ情報より商品単体のaタグ情報を取得し、商品詳細ページURLを抽出
            for li_tag_content in li_tag_contents:
                li_tag_content_string = li_tag_content.get_attribute('innerHTML')
                a_tag_url = li_tag_content_string.split('"')[1]
                product_url_list.append(a_tag_url)

        #商品詳細ページ
        #商品詳細ページへ遷移し対象情報を取得
        for product_url in product_url_list:
                        
            #商品詳細ページへのurl生成
            target_url = root + product_url
            #商品詳細ページへ遷移
            driver.get(target_url)

            #製品詳細ページから対象情報を取得
            #製品名取得
            product_name = driver.find_element(By.CLASS_NAME, "hdg")
            #製品詳細取得
            product_detail_all = driver.find_element(By.CLASS_NAME, "detail-content")
            product_detail_content = product_detail_all.text.split('\n')
            if len(product_detail_content) >= 2:
                product_detail = product_detail_content[1]
            else:
                product_detail = ""
            #原材料名取得
            product_raw_material_all = driver.find_element(By.CLASS_NAME, "column.col2.large")
            product_raw_material_content = product_raw_material_all.text.split('\n')
            if len(product_raw_material_content) >= 2:
                product_material = product_raw_material_content[1]
            else: 
                product_material = ""
    
            #JANコード、内容量、価格(テーブル)取得
            product_table = driver.find_element(By.CLASS_NAME, "table-01.headNo") #テーブルが複数あった場合は最初のテーブルを取得
            table_contents = product_table.text.split('\n')

           #辞書にブランド名・温度・スーパーマーケットを入力(外生変数)
            data_product['Brand'].append(product_brand)
            data_product['Temperature'].append(product_temperature)
            data_product['Supermarket'].append(product_supermarket)

            #辞書に商品名を入力 (テーブル以外)
            data_product['SKU_NAME_JP'].append(product_name.text)
            data_product['Detail'].append(product_detail)
            data_product['Raw_Material'].append(product_material) 

            #辞書にJANコード、内容量、価格を入力
            for table_content in table_contents:
                split_content = table_content.split(' ')   #それぞれの要素を半角スペースで区切ってリストへ格納
                if split_content[0] == 'JANコード':
                    data_product['SKU_Barcode'].append(split_content[1])
                elif split_content[0] == '内容量':
                    data_product['SKU_Weight_(g/unit)'].append(split_content[1])
                elif split_content[0] == '希望小売価格':
                    data_product['Consumer_Price'].append(split_content[1])
            
            #データが空欄だった場合は空文字を入力
            if len(data_product['SKU_Barcode']) != counter + 1:
                data_product['SKU_Barcode'].append("")
            if len(data_product['SKU_Weight_(g/unit)']) != counter + 1:
                data_product['SKU_Weight_(g/unit)'].append("")
            if len(data_product['Consumer_Price']) != counter + 1:
                data_product['Consumer_Price'].append("")
            
            #カウンターインデックスの更新
            counter += 1
            
#driverのプロセス終了
driver.quit()

In [ ]:
#スクレイピング結果の出力
result = pd.DataFrame(data_product)
result

In [ ]:
#Excelファイルとして保存
result.to_excel("result_scraping.xlsx")